In [1]:
pip install tfimm

     |████████████████████████████████| 139 kB 600 kB/s            
     |████████████████████████████████| 14.8 MB 3.8 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77059 sha256=17d8ab3f3beacb8e4ec9bd3b73b4c0f02ad7986b2099243e3ea5bbb7adb97967
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: wrapt
    Found 

In [2]:
pip install timm

     |████████████████████████████████| 431 kB 517 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tfimm
import timm
model_list = tfimm.list_models(pretrained="timm")
str_match = [s for s in model_list if "vit" in s]
str_match

['vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_huge_patch14_224_in21k',
 'vit_large_patch16_224',
 'vit_large_patch16_224_in21k',
 'vit_large_patch16_384',
 'vit_large_patch32_224_in21k',
 'vit_large_patch32_384',
 'vit_large_r50_s32_224',
 'vit_large_r50_s32_224_in21k',
 'vit_large_r50_s32_384',
 'vit_small_patch16_224',
 'vit_small_patch16_224_in21k',
 'vit_small_patch16_384',
 'vit_small_patch32_224',
 'vit_small_patch32_224_in21k',
 'vit_small_patch32_384',
 'vit_small_r26_s32_224',
 'vit_small_r26_s32_224_in21k',
 'vit_small_r26_s32_384',
 'vit_tiny_patch16_224',
 'vit_tiny_patch16_224_in21k',
 'vit_tiny_patch16_384',
 'vit_tiny_r_s

In [4]:
vit_model = tfimm.create_model("vit_base_patch32_224", pretrained="timm")
vit_model.summary()

2022-04-08 11:25:26.843660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 11:25:26.844629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 11:25:26.845311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 11:25:26.846193: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "vi_t"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
patch_embed (PatchEmbeddings multiple                  2360064   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
blocks/0 (ViTBlock)          multiple                  7087872   
_________________________________________________________________
blocks/1 (ViTBlock)          multiple                  7087872   
_________________________________________________________________
blocks/2 (ViTBlock)          multiple                  7087872   
_________________________________________________________________
blocks/3 (ViTBlock)          multiple                  7087872   
_________________________________________________________________
blocks/4 (ViTBlock)          multiple                  7087872

input shape de CIFAR-10

In [5]:
input_shape = (32, 32, 3)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.Input(shape=input_shape))
for layer in vit_model.layers[0:-1]:
    model.add(layer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
patch_embed (PatchEmbeddings (None, None, None)        2360064   
_________________________________________________________________
dropout (Dropout)            (None, None, None)        0         
_________________________________________________________________
blocks/0 (ViTBlock)          (None, None, 768)         7087872   
_________________________________________________________________
blocks/1 (ViTBlock)          (None, None, 768)         7087872   
_________________________________________________________________
blocks/2 (ViTBlock)          (None, None, 768)         7087872   
_________________________________________________________________
blocks/3 (ViTBlock)          (None, None, 768)         7087872   
_________________________________________________________________
blocks/4 (ViTBlock)          (None, None, 768)         7

hacemos las capas no entrenables

In [7]:
#for layer in model.layers:
#    layer.trainable = False
#model.summary()

añadimos la capa de salida correspondiente a nuestro problema (CIFAR-10)

In [8]:
model.add(keras.layers.Dense(10))
loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optim = keras.optimizers.Adam(learning_rate=0.0001)
metrics = ["accuracy"]

model.compile(optimizer = optim, loss=loss, metrics=metrics)

In [9]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 256
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
(x_train, y_train), (x_val, y_val) = (
    (x_train[:40000], y_train[:40000]),
    (x_train[40000:], y_train[40000:]),
)
print(f"Training samples: {len(x_train)}")
print(f"Validation samples: {len(x_val)}")
print(f"Testing samples: {len(x_test)}")

# Convert to tf.data.Dataset objects.
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(BATCH_SIZE * 100).batch(BATCH_SIZE).prefetch(AUTO)

val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTO)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTO)

170508288/170498071 [==============================] - 6s 0us/step
Training samples: 40000
Validation samples: 10000
Testing samples: 10000


In [10]:
model.fit(train_ds, validation_data = val_ds, epochs=30)

Epoch 1/30


2022-04-08 11:26:01.947286: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


157/157 [==============================] - 25s 89ms/step - loss: 2.1822 - accuracy: 0.3145 - val_loss: 1.7601 - val_accuracy: 0.3870
Epoch 2/30
157/157 [==============================] - 12s 74ms/step - loss: 1.6351 - accuracy: 0.4256 - val_loss: 1.6064 - val_accuracy: 0.4400
Epoch 3/30
157/157 [==============================] - 11s 72ms/step - loss: 1.4629 - accuracy: 0.4845 - val_loss: 1.4735 - val_accuracy: 0.4878
Epoch 4/30
157/157 [==============================] - 11s 72ms/step - loss: 1.3297 - accuracy: 0.5358 - val_loss: 1.4070 - val_accuracy: 0.5064
Epoch 5/30
157/157 [==============================] - 11s 72ms/step - loss: 1.2169 - accuracy: 0.5748 - val_loss: 1.3910 - val_accuracy: 0.5099
Epoch 6/30
157/157 [==============================] - 11s 72ms/step - loss: 1.1016 - accuracy: 0.6175 - val_loss: 1.2926 - val_accuracy: 0.5511
Epoch 7/30
157/157 [==============================] - 11s 72ms/step - loss: 1.0134 - accuracy: 0.6528 - val_loss: 1.3101 - val_accuracy: 0.5436
Epo

In [11]:
model.evaluate(test_ds)

40/40 [==============================] - 1s 32ms/step - loss: 2.0369 - accuracy: 0.6071


[2.036909341812134, 0.6071000099182129]